In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import copy
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

class WavefrontEnv:
    def __init__(self, slm_dim1=64, slm_dim2=64, eng_size=1,
                 num_pix_per_block=32, alpha=0.3, noise_sigma=0.05, k=0.45, phi=None):
        # geometry
        self.slm_dim1, self.slm_dim2 = slm_dim1, slm_dim2
        self.n_pix   = slm_dim1 * slm_dim2            
        self.eng_size = eng_size                      

        # phase mask
        if phi is None:
            self.phi = np.random.rand(slm_dim1, slm_dim2)
        else:
            assert phi.shape == (slm_dim1, slm_dim2), "Phase mask shape mismatch."
            self.phi = phi

        # block grid
        self.blocks = self._make_blocks(num_pix_per_block)
        self.num_blocks = len(self.blocks)            

        # RL bookkeeping
        self.state_dim    = self.num_blocks           
        self.action_space = self.num_blocks
        self.alpha  = alpha
        self.sigma  = noise_sigma                     
        self.I0_mean = 0.0
        self.I_max  = 0.0
        self.I_t   = 0.0

        self.k = k

        self.best_mask = None

        self.set_I0_mean()

        self.reset()

    #utils
    def _make_blocks(self, p_per_block):
        """Return list of numpy arrays, each array holds pixel indices of one block."""
        idx = np.arange(self.n_pix)
        return [idx[k : k + p_per_block]
                for k in range(0, self.n_pix, p_per_block)]

    def _blocks_to_pixels(self):
        """Convert block-level mask → 2-D pixel mask."""
        pixel = np.zeros(self.n_pix, dtype=np.float32)
        for bid, bit in enumerate(self.block_mask):
            if bit:                                     
                pixel[self.blocks[bid]] = 1.0
        return pixel.reshape(self.slm_dim1, self.slm_dim2)

    def reset(self):
        if self.best_mask is not None:
            if np.random.rand() < 0.2:
                self.block_mask = self.best_mask.copy()

                # Flip a small number of bits randomly (e.g., 5 out of 128)
                flip_indices = np.random.choice(self.num_blocks, size=5, replace=False)
                for idx in flip_indices:
                    self.block_mask[idx] = 1.0 - self.block_mask[idx]
            else:
                self.block_mask = self.best_mask.copy()
        else:
            self.block_mask = np.random.choice([0.0, 1.0], size=self.num_blocks).astype(np.float32)
            
        self.I_prev = self._intensity()
        self.I_max  = self.I_prev
        return self._state()
    
    def tanh_reward(self, I_1):
        return np.tanh(self.k * I_1)

    def step(self, action):
        
        for a in action:
            self.block_mask[a] = 1.0 - self.block_mask[a]

        self.I_t = self._intensity()
        reward = self.tanh_reward(self.I_t)
         
        # Bookkeeping best mask
        if self.I_t > self.I_max:
            self.I_max = self.I_t
            self.best_mask = self.block_mask.copy()

        return self._state(), reward

    #optics
    def _intensity(self):
        mask2d = self._blocks_to_pixels()
        field  = np.exp(1j * 2 * np.pi * self.phi) * mask2d
        spec   = np.fft.fftshift(np.fft.fft2(field))
        I      = np.abs(spec[self.slm_dim1 // 2, self.slm_dim2 // 2])**2 / spec.size
        I     += self.sigma * np.random.randn()         # additive Gaussian noise
        return float(I)

    #state
    def _state(self):
        return self.block_mask.astype(np.float32)
    
    def set_I0_mean(self):
        for i in range(1000):
            self.block_mask = np.random.choice([0.0, 1.0], size=self.num_blocks).astype(np.float32)
            I = self._intensity()
            self.I0_mean += I
        self.I0_mean /= 1000.0

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(ActorCritic, self).__init__()
        
        # Actor head
        self.actor_head = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, action_dim)
        )

        # Critic head
        self.critic_head = nn.Sequential(
            nn.Linear(state_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
    def forward(self, x):
    
        action_logits = self.actor_head(x)
        
        # Critic: State value
        state_value = self.critic_head(x)
        
        # return action_probs, state_value
        return action_logits, state_value
    
    def get_action(self, state, deterministic=False):
        state = torch.FloatTensor(state).unsqueeze(0)

        action_logits, _ = self.forward(state)         # [1, 128]
    
        dist = torch.distributions.Bernoulli(logits=action_logits)
        
        if deterministic:
            # Using threshold 0.5 to binarize
            action = (torch.sigmoid(action_logits) > 0.5).float()
        else:
            action = dist.sample()
        
        # Convert to list of indices where action is 1
        action = action.squeeze(0).detach().cpu().numpy().astype(int)
        flip_indices = np.where(action == 1)[0].tolist()
        
        return flip_indices, action
    
    def evaluate(self, states, actions):
        action_logits, state_values = self.forward(states)
        dist = torch.distributions.Bernoulli(logits=action_logits)
        
        # Element-wise log-probabilities: shape [T, 128]
        action_log_probs = dist.log_prob(actions)

        # Sum over action dimensions to get [T]
        log_probs = action_log_probs.sum(dim=1)

        # Entropy per sample (sum over 128 dimensions): shape [T]
        dist_entropy = dist.entropy().sum(dim=1)

        return log_probs, state_values, dist_entropy

# PPO Agent
class PPOAgent:
    def __init__(self, state_dim, action_dim, lr=3e-4, actor_lr=3e-4, critic_lr=3e-4, gamma=0.99, 
                 clip_ratio=0.2, value_coef=0.5, entropy_coef=0.01, gae_lambda=0.95):
        self.gamma = gamma
        self.clip_ratio = clip_ratio
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        self.gae_lambda = gae_lambda
        
        self.policy = ActorCritic(state_dim, action_dim)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.actor_optimizer = optim.Adam(self.policy.actor_head.parameters(), lr=actor_lr)
        self.critic_optimizer = optim.Adam(self.policy.critic_head.parameters(), lr=critic_lr)
        
        self.old_policy = copy.deepcopy(self.policy)
        self.old_policy.eval()  # Set to evaluation mode
    
    def update(self, states, actions, rewards, next_states, steps_per_epoch=128, epochs=10):
        # Convert to tensors
        states = torch.FloatTensor(np.array(states))
        actions = torch.FloatTensor(np.array(actions))
        rewards = torch.FloatTensor(np.array(rewards))
        next_states = torch.FloatTensor(np.array(next_states))

        kl_divergences = []

        with torch.no_grad():
            old_action_log_probs, old_values, _ = self.old_policy.evaluate(states, actions)
            old_values = old_values.squeeze(-1)
            next_values = self.old_policy.forward(next_states)[1].squeeze(-1)
            values = torch.cat([old_values, next_values[-1].unsqueeze(0)], dim=0)

        # Calculate advantages using GAE (Generalized Advantage Estimation)
        advantages = torch.zeros_like(rewards)
        returns = torch.zeros_like(rewards)
        gae = 0

        for t in reversed(range(len(rewards))):
            next_value = values[t + 1]
            delta = rewards[t] + self.gamma * next_value - old_values[t]
            gae = delta + self.gamma * self.gae_lambda * gae
            advantages[t] = gae
        
        # Compute returns (used for value function loss)
        returns = advantages + old_values.detach()
        
        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        # PPO update loop
        for _ in range(epochs):
            # Create random indices for minibatches
            indices = np.random.permutation(len(states))
            
            # Iterate through mini-batches
            for start in range(0, len(states), steps_per_epoch):
                end = start + steps_per_epoch
                if end > len(states):
                    end = len(states)
                    
                batch_indices = indices[start:end]
                
                # Get batch data
                batch_states = states[batch_indices]
                batch_actions = actions[batch_indices]
                batch_advantages = advantages[batch_indices]
                batch_returns = returns[batch_indices]
                batch_old_action_log_probs = old_action_log_probs[batch_indices]
                
                # Evaluate current policy
                action_log_probs, values, entropy = self.policy.evaluate(batch_states, batch_actions)
                
                # Calculate ratios
                ratios = torch.exp(action_log_probs - batch_old_action_log_probs)

                kl = (batch_old_action_log_probs - action_log_probs).mean().item()  
                kl_divergences.append(kl)
                
                # Compute surrogate losses
                surr1 = ratios * batch_advantages
                surr2 = torch.clamp(ratios, 1.0 - self.clip_ratio, 1.0 + self.clip_ratio) * batch_advantages
                
                # Calculate loss components
                policy_loss = -torch.min(surr1, surr2).mean()
                # value_loss
                value_loss = F.smooth_l1_loss(values.squeeze(-1), batch_returns)
                entropy_loss = -entropy.mean()

                actor_loss = policy_loss + self.entropy_coef * entropy_loss
                critic_loss = self.value_coef * value_loss

                # Update actor and critic
                self.actor_optimizer.zero_grad()
                actor_loss.backward()
                nn.utils.clip_grad_norm_(self.policy.actor_head.parameters(), 0.5)
                self.actor_optimizer.step()

                self.critic_optimizer.zero_grad()
                critic_loss.backward()
                nn.utils.clip_grad_norm_(self.policy.critic_head.parameters(), 0.5)
                self.critic_optimizer.step()
        
        with torch.no_grad():
            new_values = self.policy.forward(states)[1].squeeze(-1)

        # Update old policy
        self.old_policy.load_state_dict(self.policy.state_dict())

        average_kl = np.mean(kl_divergences)
        
        return policy_loss.item(), value_loss.item(), entropy_loss.item(), advantages, old_values, new_values, returns, average_kl
    
    def get_action(self, state, deterministic=False):
        return self.policy.get_action(state, deterministic)
    
    def save(self, path):
        torch.save(self.policy.state_dict(), path)
        
    def load(self, path):
        self.policy.load_state_dict(torch.load(path))
        self.old_policy.load_state_dict(torch.load(path))

# Training function
def train_ppo(env, agent, max_episodes=1000, steps_per_episode=128, 
              update_interval=10, eval_interval=50, verbose=True):
    
    # Initialize logging variables
    episode_rewards = []
    eval_rewards = []
    best_intensity = 0
    best_mask = None

    # Storage for the all states, actions, rewards, and next_states
    states = []
    actions = []
    rewards = []
    next_states = []

    all_advantage_means = []

    value_losses = []
    policy_losses = []
    entropy_losses = []
    all_old_values = []
    all_new_values = []
    all_returns = []
    all_kl_divergences = []
    
    # Training loop
    for episode in tqdm(range(1, max_episodes + 1)):
        
        episode_reward = 0
        state = env.reset()
        
        # Generate trajectory
        for step in range(steps_per_episode):
            # Select action
            action, action_mask = agent.get_action(state)
            
            # Execute action
            next_state, reward = env.step(action)
            
            # Store transition
            states.append(state)
            actions.append(action_mask)
            rewards.append(reward)
            next_states.append(next_state)
            
            # Update state and reward
            state = next_state
            episode_reward += reward
            
        # Store episode reward
        episode_rewards.append(episode_reward)
        
        # Update agent
        if episode % update_interval == 0:
            policy_loss, value_loss, entropy_loss, advantages, old_values, new_values, returns, kl_divergence = agent.update(
                states, actions, rewards, next_states
            )
            
            advantages = advantages.detach().cpu().numpy().reshape(update_interval, steps_per_episode)
            per_episode_advantage_means = advantages.mean(axis=1)

            all_advantage_means.extend(per_episode_advantage_means)

            # Append scalar value loss
            value_losses.append(value_loss)
            policy_losses.append(policy_loss)
            entropy_losses.append(entropy_loss)

            # Detach and move tensors to CPU
            all_old_values.append(old_values.detach().cpu().numpy())
            all_new_values.append(new_values.detach().cpu().numpy())
            all_returns.append(returns.detach().cpu().numpy())
            all_kl_divergences.append(kl_divergence)

            if verbose:
                print(f"Episode {episode}, Reward: {episode_reward:.4f}")
                print(f"Policy Loss: {policy_loss:.4f}, Value Loss: {value_loss:.4f}, Entropy Loss: {entropy_loss:.4f}")
                print(f"Current Intensity: {env.I_prev:.4f}, Max Intensity: {env.I_max:.4f}")
                print("---")
            
            states.clear()
            actions.clear()
            rewards.clear()
            next_states.clear()
        
        # Track best solution found
        if env.I_max > best_intensity:
            best_intensity = env.I_max
            best_mask = env.block_mask.copy()
        
        # Evaluate performance
        if episode % eval_interval == 0:
            eval_reward = evaluate_agent(env, agent, num_episodes=5)
            eval_rewards.append(eval_reward)
            
            if verbose:
                print(f"Evaluation at episode {episode}: Average Reward = {eval_reward:.4f}")
                print("===================================")
    
    old_values_all = np.concatenate(all_old_values)
    new_values_all = np.concatenate(all_new_values)
    returns_all = np.concatenate(all_returns)

    # Return results
    return {
        'episode_rewards': episode_rewards,
        'eval_rewards': eval_rewards,
        'best_intensity': best_intensity,
        'best_mask': best_mask,
        'all_advantage_means': all_advantage_means,
        'value_losses': value_losses,
        'old_values_all': old_values_all,
        'new_values_all': new_values_all,
        'returns_all': returns_all,
        'policy_losses': policy_losses,
        'entropy_losses': entropy_losses,
        'all_kl_divergences': all_kl_divergences,
    }

# Evaluation function
def evaluate_agent(env, agent, num_episodes=5, steps_per_episode=128):
    total_rewards = 0
    
    for _ in range(num_episodes):
        state = env.reset()
        episode_reward = 0
        
        for _ in range(steps_per_episode):
            action, _ = agent.get_action(state, deterministic=True)  # Use deterministic policy for evaluation
            next_state, reward = env.step(action)
            
            state = next_state
            episode_reward += reward
                
        total_rewards += episode_reward
        
    return total_rewards / num_episodes

# Visualization functions 
def plot_training_curve(results, run): 

    plt.figure(figsize=(16, 16))

    # Plot episode rewards
    plt.subplot(7, 1, 1)
    plt.plot(results['episode_rewards'], label='Episode Reward')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.title('Training Rewards')
    plt.grid(True)
    plt.legend()
    
    # Plot evaluation rewards if available
    if len(results['eval_rewards']) > 0:
        plt.subplot(7, 1, 2)
        eval_x = np.linspace(0, len(results['episode_rewards']), len(results['eval_rewards']))
        plt.plot(eval_x, results['eval_rewards'], label='Evaluation Reward', color='orange')
        plt.xlabel('Episode')
        plt.ylabel('Average Reward')
        plt.title('Evaluation Rewards')
        plt.grid(True)
        plt.legend()

    # Plot mean advantage per episode
    if 'all_advantage_means' in results and len(results['all_advantage_means']) > 0:
        plt.subplot(7, 1, 3)
        update_x = np.linspace(0, len(results['episode_rewards']), len(results['all_advantage_means']))
        plt.plot(update_x, results['all_advantage_means'], label='Mean Advantage', color='green')
        plt.xlabel('Episode')
        plt.ylabel('Mean Advantage')
        plt.title('Advantage Trend Over Updates')
        plt.grid(True)
        plt.legend()

    # Plot value loss over updates
    if 'value_losses' in results and len(results['value_losses']) > 0:
        plt.subplot(7, 1, 4)
        plt.plot(results['value_losses'], label='Value Loss', color='red')
        plt.xlabel('Update Step')
        plt.ylabel('Value Loss')
        plt.title('Value Loss Trend Over Updates')
        plt.grid(True)
        plt.legend()

    # Plot policy loss over updates
    if 'policy_losses' in results and len(results['policy_losses']) > 0:
        plt.subplot(7, 1, 5)
        plt.plot(results['policy_losses'], label='Policy Loss', color='blue')
        plt.xlabel('Update Step')
        plt.ylabel('Policy Loss')
        plt.title('Policy Loss Trend Over Updates')
        plt.grid(True)
        plt.legend()

    # Plot entropy loss over updates
    if 'entropy_losses' in results and len(results['entropy_losses']) > 0:
        plt.subplot(7, 1, 6)
        plt.plot(results['entropy_losses'], label='Entropy Loss', color='purple')
        plt.xlabel('Update Step')
        plt.ylabel('Entropy Loss')
        plt.title('Entropy Loss Trend Over Updates')
        plt.grid(True)
        plt.legend()
    
    # Plot KL divergence over updates
    if 'all_kl_divergences' in results and len(results['all_kl_divergences']) > 0:
        plt.subplot(7, 1, 7)
        plt.plot(results['all_kl_divergences'], label='KL Divergence', color='brown')
        plt.xlabel('Update Step')
        plt.ylabel('KL Divergence')
        plt.title('KL Divergence Trend Over Updates')
        plt.grid(True)
        plt.legend()

    plt.tight_layout()
    plt.savefig(f'results/run_{run}_training_curve.png')
    plt.close()

    # Plot value estimates vs returns
    if 'old_values_all' in results and 'returns_all' in results:
        plt.figure(figsize=(8, 6))
        plt.scatter(results['returns_all'], results['old_values_all'], alpha=0.5, label='Old Value Estimates')
        plt.scatter(results['returns_all'], results['new_values_all'], alpha=0.5, label='New Value Estimates')
        plt.plot(results['returns_all'], results['returns_all'], 'k--', label='Ideal Match')
        plt.xlabel('Returns')
        plt.ylabel('Value Estimates')
        plt.title('Value Estimates vs Returns')
        plt.legend()
        plt.grid(True)
        plt.savefig(f'results/run_{run}_value_vs_returns.png')
        plt.close()


def visualize_best_mask(env, best_mask, run):
    # Store original mask
    original_mask = env.block_mask.copy()
    
    # Set the best mask
    env.block_mask = best_mask.copy()
    
    # Get pixel mask
    pixel_mask = env._blocks_to_pixels()
    
    # Get intensity
    intensity = env._intensity()
    
    # Plot mask
    plt.figure(figsize=(10, 8))
    plt.imshow(pixel_mask, cmap='viridis')
    plt.colorbar(label='Mask Value')
    plt.title(f'Best Mask (Intensity: {intensity:.4f})')
    plt.savefig(f'results/run_{run}_best_mask.png')
    plt.close()
    
    # Restore original mask
    env.block_mask = original_mask

def main(run):
    np.random.seed(42)
    
    phi_path = "phase_mask/phi_64.npy"

    if os.path.exists(phi_path):
        phi = np.load(phi_path)
    else:
        phi = np.random.rand(64, 64)
        os.makedirs(os.path.dirname(phi_path), exist_ok=True)
        np.save(phi_path, phi)
    

    # Create environment
    env = WavefrontEnv(slm_dim1=64, slm_dim2=64, num_pix_per_block=32, alpha=0.3, noise_sigma=0.05, k=0.45, phi=phi)
    
    # Create PPO agent
    agent = PPOAgent(
        state_dim=env.state_dim,
        action_dim=env.action_space,
        lr=3e-4,
        actor_lr=3e-4,
        critic_lr=1e-4,
        gamma=0.95,
        clip_ratio=0.2,
        value_coef=0.7,
        entropy_coef=0.001,
        gae_lambda=0.95
    )
    
    # Train agent
    print("Starting training...")
    results = train_ppo(
        env=env,
        agent=agent,
        max_episodes=3000,  # Adjust based on your time constraints
        steps_per_episode=128,
        update_interval=5,
        eval_interval=25
    )
    
    # Visualize results
    plot_training_curve(results, run)
    visualize_best_mask(env, results['best_mask'], run)
    
    print(f"Training completed. Best intensity achieved: {results['best_intensity']:.4f}")


main(1)

Starting training...


  1%|          | 5/500 [00:01<03:39,  2.26it/s]

Episode 5, Reward: 17.1293
Policy Loss: -0.0118, Value Loss: 0.0716, Entropy Loss: -88.6023
Current Intensity: 1.7904, Max Intensity: 1.7904
---


  2%|▏         | 10/500 [00:03<03:41,  2.21it/s]

Episode 10, Reward: 15.2031
Policy Loss: -0.2264, Value Loss: 0.0446, Entropy Loss: -88.5742
Current Intensity: 2.2056, Max Intensity: 2.2056
---


...

100%|█████████▉| 499/500 [04:39<00:00,  1.71it/s]

Episode 500, Reward: 22.8180
Policy Loss: -0.3219, Value Loss: 0.0555, Entropy Loss: -84.9236
Current Intensity: 5.1715, Max Intensity: 5.1715
---


100%|██████████| 500/500 [04:44<00:00,  1.76it/s]

Evaluation at episode 500: Average Reward = 109.7120


Training completed. Best intensity achieved: 5.5385
